In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
import math
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')
from shapely.geometry import LineString
from shapely.geometry import Point
from math import radians, cos, sin, asin, sqrt

In [2]:
path=('./valid/')

In [3]:
DATA=pd.read_csv(path+'For_matrix_TB_2019-10-07.csv', sep = ',', encoding='utf-8')
DATA2=pd.read_csv(path+'For_matrix_TB_2019-10-08.csv', sep = ',', encoding='utf-8')
DATA2['time']=DATA2['time']+24*3600 #прибалвяем 24 часа в секундах
DATA3=pd.read_csv(path+'For_matrix_TB_2019-10-09.csv', sep = ',', encoding='utf-8')
DATA3['time']=DATA3['time']+24*3600+24*3600
DATA=DATA.append(DATA2).append(DATA3)

In [4]:
DATA=DATA[['Дата и время поездки','Номер кристалла','TCAT','Код Маршрута','Код предприятия_x','mr_id','time','ssId','rl_id','rc_orderby']]

In [5]:
len(DATA)

29332

In [6]:
del DATA2
del DATA3

Привязали все валидации к сайтам

In [7]:
sites=pd.read_csv('stop_seq.csv', sep = ',', encoding='utf-8')

In [8]:
sites[0:2]

,ssId,rc_orderby,ss_lat,ss_long,rl_id,mr_id,site_id
0,stop__9854427,1,53.557965,49.228861,2227623047,2227622934,14
1,stop__9854425,2,53.557324,49.236213,2227623047,2227622934,14


In [9]:
sites=sites[['ssId','site_id']].drop_duplicates()
DATA=DATA.merge(sites,on='ssId', how='left')

In [10]:
DATA[0:2]

,Дата и время поездки,Номер кристалла,TCAT,Код Маршрута,Код предприятия_x,mr_id,time,ssId,rl_id,rc_orderby,site_id
0,2019-10-07 19:03:24,F5756324,ТКУ,42303456789,26,2227622944,68604,stop__9854571,2.278607e+09,11.0,154.0
1,2019-10-07 19:03:26,1C3A4AF9,ТКУ,42303456789,26,2227622944,68606,stop__9854571,2.278607e+09,11.0,154.0


ВЫБРАЛИ ДАННЫЕ ЗА ЧАС ПИК

In [11]:
#выбрала час пик только из перовго дня
DATA_pick=DATA[DATA['Дата и время поездки'].str.contains( '2019-10-07')] 
DATA_pick=DATA_pick[DATA_pick['Дата и время поездки'].str.contains( ' 06:')]\
    .append(DATA_pick[DATA_pick['Дата и время поездки'].str.contains( ' 07:')])\
    .append(DATA_pick[DATA_pick['Дата и время поездки'].str.contains( ' 08:')])

In [12]:
len(DATA_pick)

2246

Убираем и запоминаем из выборки тех, у кого нет номера кристалла (невозможно точно привязать точку выхода)

In [13]:
DATA_bank_pick=DATA_pick[DATA_pick['Номер кристалла'].isna()]

ОСТАВЛЯЕМ В ЧАСЕ ПИК ТОЛЬКО ТЕХ, у кого есть номер кристалла:

In [14]:
DATA_pick=DATA_pick[~DATA_pick.index.isin(DATA_bank_pick.index)]

In [15]:
DATA_pick['time'].min()

21609

In [16]:
DATA_pick['time'].max()

32355

Выбрала из общего массива только те данные, где встречаются валидации из пикового промежутка:

In [17]:
DATA=DATA[DATA['Номер кристалла'].isin(DATA_pick['Номер кристалла'].unique())]

In [18]:
len(DATA)

7228

Выбрала только те значения, которые встречаются спустя 3 часа после начала часа пик

In [19]:
DATA=DATA[DATA['time']>DATA_pick['time'].min()+3*3600]

In [20]:
DATA=DATA.sort_values(['Номер кристалла','time','site_id'])

In [21]:
DATA_pick=DATA_pick.sort_values(['Номер кристалла','time','site_id'])

In [22]:
len(DATA_pick)

2089

In [23]:
len(DATA)

5134

In [24]:
from tqdm import tqdm
k=0
res=DATA_pick
result = []
result2 = []
result3 = []
result4 = []
stops=pd.DataFrame()
with tqdm(total=DATA_pick.shape[0]) as pbar: 
    for index, row in DATA_pick.iterrows():
        pbar.update(1)
        stops=DATA[(DATA['Номер кристалла']==row['Номер кристалла'])
                    &(DATA['time']>(row['time']+3600*3))
                    &(DATA['site_id']!=row['site_id'])]
        if stops.empty:
            result.append(0)
            result2.append(0)
            result3.append(0)
            result4.append(0)
        else:
            stop_unic=stops['site_id'].values
            result.append(stop_unic[0])
            time_2=stops['time'].values
            result2.append(time_2[0])
            rl_id=stops['rl_id'].values
            result3.append(rl_id[0])
            rc_orderby=stops['rc_orderby'].values
            result4.append(rc_orderby[0])
res['site_id_2']=result
res['time_2']=result2
res['rl_id_2']=result3
res['rc_orderby_2']=result4

100%|█████████████████████████████████████████████████████████████████████████████| 2089/2089 [00:04<00:00, 497.29it/s]


In [25]:
res.to_csv('res_2019_10_07.csv')

In [26]:
#res=pd.read_csv('res_2019_10_16.csv')

In [27]:
len(res)

2089

In [28]:
len(DATA_bank_pick)

157

In [29]:
len(res[(res['site_id_2']>0)&(res['site_id']>0)&(res['site_id_2']!=res['ssId'])])

1450

In [30]:
len(res[(res['site_id_2']>0)&(res['site_id']>0)&(res['site_id_2']!=res['ssId'])])/len(res)

0.6941120153183341

In [31]:
res=res.fillna(0)

In [32]:
res['c']=1

In [33]:
res['c'].sum()

2089

In [34]:
res=res.groupby(['site_id','site_id_2','mr_id']).agg({'c':np.nansum}).reset_index() 

In [35]:
res['c'].sum()

2089

Заполняем неприязавшиеся Site_ID наиболее подходящими ID сайтов на маршруте

In [36]:
res_not_null=res[(res['site_id_2']>0)&(res['site_id']>0)] #выбрали из матрицы те строки, где все значения есть
res_null_last_stops=res[((res['site_id_2']==0)|(res['site_id_2'].isna()))
                        &(res['site_id']>0)].drop(['site_id_2'], axis=1) #выбрали только те строки, где есть остановка входа, но нет остановки выхода
res_null_first_stops=res[((res['site_id']==0)|(res['site_id'].isna()))
                        &(res['site_id_2']>0)].drop(['site_id'], axis=1) #выбрали только те строки, где есть остановка вЫхода, но нет остановки входа

Записываем всё, что не записалось и не определилось в массив res_null_null

In [37]:
res_null_null=res[~res.index.isin(res_not_null.index)]
res_null_null2=res_null_null[~res_null_null.index.isin(res_null_last_stops.index)]
del res_null_null
res_null_null=res_null_null2[~res_null_null2.index.isin(res_null_first_stops.index)].drop(['site_id'], axis=1).drop(['site_id_2'], axis=1)
del res_null_null2


In [38]:
res_null_null=res_null_null.reset_index()
res_null_last_stops=res_null_last_stops.reset_index()
res_null_first_stops=res_null_first_stops.reset_index()

In [39]:
len(res)-(len(res_not_null)+len(res_null_null)+len(res_null_last_stops)+len(res_null_first_stops))

0

In [40]:
res['c'].dropna().sum()

2089

In [41]:
res_null_null['c'].unique()

array([ 2,  8, 32], dtype=int64)

Добавляем банк

In [42]:
#добавили в выборку, где не привязались посл.остановки баноквские карты, где есть валидация начала
DATA_bank_ssid=DATA_bank_pick[DATA_bank_pick['site_id']>0]
DATA_bank_ssid['c']=1
DATA_bank_ssid=DATA_bank_ssid[['mr_id','site_id','c']]
res_null_last_stops=res_null_last_stops.append(DATA_bank_ssid).reset_index()

In [43]:
#добавили в выборку, где ничего не привязалось баноквские карты, где нет валидаций начала
DATA_bank_null=DATA_bank_pick[~DATA_bank_pick.index.isin(DATA_bank_ssid.index)]
DATA_bank_null['c']=1
DATA_bank_null=DATA_bank_null[['mr_id','site_id','c']]
res_null_null=res_null_null.append(DATA_bank_null).reset_index()

In [44]:
len(res)+len(DATA_bank_pick)-(len(res_not_null)+len(res_null_null)+len(res_null_last_stops)+len(res_null_first_stops))

0

Сгруппировали все выгрузки, чтобы меньше хранить в оперативной памяти:

In [45]:
res_null_null=res_null_null.fillna(0)

In [46]:
res_null_null=res_null_null.groupby(['mr_id']).agg({'c':np.sum}).reset_index() 

In [47]:
res_null_last_stops=res_null_last_stops.groupby(['mr_id','site_id']).agg({'c':np.sum}).reset_index() 

In [48]:
res_null_first_stops=res_null_first_stops.groupby(['mr_id','site_id_2']).agg({'c':np.sum}).reset_index() 

In [49]:
res_not_null=res_not_null.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index() 

In [50]:
res_not_null=res_not_null.sort_values(by=['c'],ascending=False)

In [51]:
res_null_null['c'].sum()+res_null_last_stops['c'].sum()+res_null_first_stops['c'].sum()+res_not_null['c'].sum()

2246

In [52]:
res_null_last_stops['c'].sum()

625

In [53]:
res_not_null['c'].sum()

1450

Определяем наиболее подходящие остановки для высадки на маршруте:

In [54]:
#Размазываем вход-выход для случая, когда мы знаем ВХОД
def input_func(res_not_null):
    input_=res_not_null.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index() 
    sum_mr_id=input_.groupby(['site_id']).agg({'c':np.sum}).reset_index() 
    input_=input_.merge(sum_mr_id, on =['site_id'])
    input_['c_x']=input_['c_x']/input_['c_y']
    input_=input_.drop(['c_y'], axis=1).rename(columns={'c_x':'c'})
    return input_

In [55]:
#Размазываем вход-выход для случая, когда мы знаем ВЫХОД
def exit_func(res_not_null):
    exit=res_not_null.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index() 
    sum_mr_id=exit.groupby(['site_id_2']).agg({'c':np.sum}).reset_index() 
    exit=exit.merge(sum_mr_id, on =['site_id_2'])
    exit['c_x']=exit['c_x']/exit['c_y']
    exit=exit.drop(['c_y'], axis=1).rename(columns={'c_x':'c'})
    return exit

In [56]:
#Там где не определился конец поездки, выбираем наиболее подходящую SsID для выхода (пропорционально выходу)
input_=input_func(res_not_null)
res_null_last_stops2=res_null_last_stops.merge(input_,on=['site_id'],how='left')
res_null_last_stops2['c']=res_null_last_stops2['c_x']*res_null_last_stops2['c_y']
res_null_last_stops2=res_null_last_stops2.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index()

In [57]:
#записываем результат
res_null_last_stops2=res_null_last_stops2.fillna(0)
res_not_null=res_not_null.append(res_null_last_stops2[res_null_last_stops2['site_id_2']>0])

In [58]:
#записываем оставшиеся непривязавшиеся остановки
res_null_last_stops3=res_null_last_stops2[res_null_last_stops2['site_id_2']==0]
res_null_null=res_null_null.append(res_null_last_stops3.drop(['site_id_2'], axis=1).drop(['site_id'], axis=1))
del res_null_last_stops2
del res_null_last_stops

In [59]:
#Там где не определился конец поездки, выбираем наиболее подходящую SsID для выхода (пропорционально входу)
exit=exit_func(res_not_null)
res_null_first_stops2=res_null_first_stops.merge(exit,on=['site_id_2'],how='left')
res_null_first_stops2['c']=res_null_first_stops2['c_x']*res_null_first_stops2['c_y']
res_null_first_stops2=res_null_first_stops2.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index()
res_null_first_stops2=res_null_first_stops2.fillna(0)
#записываем результат
res_not_null=res_not_null.append(res_null_first_stops2[res_null_first_stops2['site_id_2']>0])

In [60]:
#записываем оставшиеся непривязавшиеся остановки
res_null_first_stops3=res_null_first_stops2[res_null_first_stops2['site_id_2']==0]
res_null_null=res_null_null.append(res_null_first_stops3.drop(['site_id_2'], axis=1).drop(['site_id'], axis=1))
del res_null_first_stops2
del res_null_first_stops
del res_null_first_stops3

Снова группируем

In [61]:
res_not_null=res_not_null.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index() 

In [62]:
res_not_null['c'].sum()

2190.0

In [63]:
#Размазываем вход-выход для случая, когда мы НИЧЕГО НЕ ЗНАЕМ
exit_input=res_not_null.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index() 
sum_mr_id=exit_input.groupby(['mr_id']).agg({'c':np.sum}).reset_index() 
exit_input=exit_input.merge(sum_mr_id, on ='mr_id')
exit_input['c_x']=exit_input['c_x']/exit_input['c_y']
exit_input=exit_input.drop(['c_y'], axis=1).rename(columns={'c_x':'c'})

In [64]:
exit_input

,mr_id,site_id,site_id_2,c
0,2227622944,17.0,52.0,0.002314
1,2227622944,17.0,53.0,0.000344
2,2227622944,17.0,62.0,0.002578
3,2227622944,17.0,87.0,0.002344
4,2227622944,17.0,88.0,0.000264
...,...,...,...,...
1490,2488443866,192.0,108.0,0.000892
1491,2488443866,193.0,53.0,0.000632
1492,2488443866,193.0,62.0,0.000737
1493,2488443866,193.0,106.0,0.001785


In [65]:
#Там где ничего не определилось, размазываем валидацию по всем парам вероятным :
res_null_null2=res_null_null.merge(exit_input,on=['mr_id'],how='left')

In [66]:
res_null_null2

,mr_id,c_x,site_id,site_id_2,c_y
0,2227622944,3.0,17.0,52.0,0.002314
1,2227622944,3.0,17.0,53.0,0.000344
2,2227622944,3.0,17.0,62.0,0.002578
3,2227622944,3.0,17.0,87.0,0.002344
4,2227622944,3.0,17.0,88.0,0.000264
...,...,...,...,...,...
984,2488443866,2.0,192.0,108.0,0.000892
985,2488443866,2.0,193.0,53.0,0.000632
986,2488443866,2.0,193.0,62.0,0.000737
987,2488443866,2.0,193.0,106.0,0.001785


In [67]:
res_null_null2['c']=res_null_null2['c_x']*res_null_null2['c_y']

In [68]:
res_null_null2=res_null_null2.drop('c_x',axis=1).drop('c_y',axis=1)

In [69]:
res_not_null['c'].sum()

2190.0

In [70]:
res_not_null2=res_not_null.append(res_null_null2)

In [71]:
res_null_null2=res_null_null2.fillna(0)

In [72]:
res_not_null2=res_not_null2.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index()

In [73]:
itog=res_not_null2['c'].sum()

In [74]:
itog

2244.0

In [75]:
#DAY=pd.read_csv(path+'For_matrix_2019_10_16.csv', sep = ',', encoding='utf-8')

Кол-во валидаций на НГПТ за день составило:

In [76]:
#day=len(DAY)

In [77]:
#day

Кол-во валидаций на НГПТ за час пик день составило:

In [78]:
#DAY_pick=DAY[DAY['Дата и время поездки'].str.contains( ' 07:')]\
#    .append(DAY[DAY['Дата и время поездки'].str.contains( ' 08:')])\
#    .append(DAY[DAY['Дата и время поездки'].str.contains( ' 09:')])
#pick=len(DAY_pick)

In [79]:
#pick
#Годовой поток по пригородным маршрутам составил 15369619
#Переход к месяцу Октябрь от года - 10,73 (делим)
#Переход к одному дню октября = 396 889 (средний будний день в октябре 2019 года)/10 852 053 (суммарный месячный поток в октябре 2019 за год)
#Переход к часу - пик - делим кол-во валидаций за день к количеству валидаций за час пик

In [80]:
#res_not_null2['c']=res_not_null2['c']+(15369619/10.73)*(396889/10852053)*(pick/day)/itog

In [81]:
#res_not_null2['c'].sum()

In [82]:
matrix=res_not_null2.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index()

In [83]:
matrix=matrix.sort_values(by='c',ascending=False)

In [84]:
matrix['site_id']=matrix['site_id'].astype('int64')

In [85]:
matrix['c']=round(matrix['c'])

In [86]:
matrix['c']=matrix['c'].astype(int)

In [87]:
matrix['site_id_2']=matrix['site_id_2'].astype('int64')

In [88]:
matrix['c'].sum()

2223

In [89]:
matrix.to_csv('./res/matrix_oct_TB.csv', index=False)

In [90]:
matrix2=pd.read_csv('./res/matrix_sep_patp.csv')

In [130]:
matrix_all=matrix.append(matrix2)

In [131]:
matrix_all=pd.DataFrame(matrix_all.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index())

In [123]:
matrix_all.to_csv('./res/matrix_Tolyatti.csv', index=False)

## Блок проверок и визуализации

## ДОБАВЛЯЕМ ВИДЕОАНАЛИТИКУ

In [323]:
#считали данные видеоаналитики
video_data=pd.read_csv('Videoanal.csv',encoding='utf-8', sep=";")
video_data=video_data[video_data['Широта'].notna()]
video_data=video_data.fillna(0)
video_data['Открытие двери']=video_data['Открытие двери'].astype(str)


In [324]:
video_data1=video_data[video_data['Выход на маршрут'].str.startswith('06')].\
append(video_data[video_data['Выход на маршрут'].str.startswith('07')]).\
append(video_data[video_data['Выход на маршрут'].str.startswith('08')])

In [325]:
video_data1=video_data1.merge(sites,left_on='ID остановки (Яндекс)',right_on='ssId',how='left')

In [326]:
#перешли к входу среднему на одном рейсе
video_data2=pd.DataFrame(video_data1.groupby(['route_id','trip_id','ID остановки (Яндекс)','site_id']).agg({'Вошло':np.mean}).reset_index())

In [327]:
#cчитали частоту хождения по маршрутам
freq=pd.read_csv('freq2.csv',encoding='utf-8',sep=';')

In [328]:
freq=freq.fillna(0)

In [329]:
freq[0:2]

,site_id,freq
0,148,58.0
1,3,58.0


In [330]:
#перешли к входу среднему на одном рейсе
video_data2=pd.DataFrame(video_data2.groupby(['trip_id','site_id']).agg({'Вошло':np.mean}).reset_index())

In [331]:
video_data2[0:2]

,trip_id,site_id,Вошло
0,2227622927,14.0,1.6
1,2227622927,19.0,1.0


In [332]:
#перешли к среднему выходу на одном рейсе автобусов 
vyxod_reis=pd.DataFrame(video_data1.groupby(['route_id','trip_id','ID остановки (Яндекс)','site_id']).agg({'Вышло':np.mean}).reset_index())

In [333]:
vyxod_reis=pd.DataFrame(vyxod_reis.groupby(['trip_id','site_id']).agg({'Вышло':np.sum}).reset_index())

In [334]:
#выбрали только те остановки,по которым будем размазывать выход (убрали те остановки, на которых у нас вход и там, где выход = 0)
#vyxod_reis=vyxod_reis[~vyxod_reis['site'].isin(video_data2['site'].unique())]
vyxod_reis=vyxod_reis[vyxod_reis['Вышло']>0]

In [335]:
#считаем суммарный выход на одном направлении на одном маршруте:
vyxod_reis_group_trip_id=pd.DataFrame(vyxod_reis.groupby(['trip_id']).agg({'Вышло':np.sum}).reset_index())

In [336]:
#размазываем 
vyxod_reis=vyxod_reis.merge(vyxod_reis_group_trip_id, on ='trip_id',how='outer')
vyxod_reis['part']=vyxod_reis['Вышло_x']/vyxod_reis['Вышло_y']
vyxod_reis=vyxod_reis[['trip_id','site_id','part']]

In [337]:
matrix_video=video_data2.merge(vyxod_reis, on ='trip_id', how='outer')

In [338]:
matrix_video['c']=matrix_video['Вошло']*matrix_video['part']

In [339]:
matrix_video=pd.DataFrame(matrix_video.groupby(['site_id_x','site_id_y']).agg({'c':np.sum}).reset_index())

In [340]:
matrix_video=matrix_video[matrix_video['site_id_x']!=matrix_video['site_id_y']]

In [341]:
matrix_video[0:2]

,site_id_x,site_id_y,c
1,0.0,2.0,0.007538
2,0.0,8.0,0.002513


In [342]:
matrix_video=matrix_video.rename(columns={'site_id_x':'site_id','site_id_y':'site_id_2'})

In [343]:
freq[0:2]

,site_id,freq
0,148,58.0
1,3,58.0


In [344]:
matrix_video=matrix_video.merge(freq,on ='site_id', how='left')

In [345]:
matrix_video['c']=matrix_video['c']*matrix_video['freq']

In [346]:
matrix_video['c'].sum()

15873.519993085485

In [347]:
matrix_all['c'].sum()

21858

In [348]:
matrix_all[0:1]

,site_id,site_id_2,c
0,2,3,6


In [349]:
matrix_video[0:1]

,site_id,site_id_2,c,freq
0,0.0,2.0,0.284659,37.764706


In [350]:
matrix_video=matrix_video[['site_id','site_id_2','c']]

In [352]:
matrix_all=matrix_all.append(matrix_video)

In [359]:
matrix_all_with_video=pd.DataFrame(matrix_all.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index())

In [360]:
matrix_all_with_video['c'].sum()

37731.51999308549

In [364]:
matrix_all_with_video['c']=np.ceil(matrix_all_with_video['c'])

In [366]:
matrix_all_with_video['c']=matrix_all_with_video['c'].astype('int64')
matrix_all_with_video['site_id']=matrix_all_with_video['site_id'].astype('int64')
matrix_all_with_video['site_id_2']=matrix_all_with_video['site_id_2'].astype('int64')

In [370]:
matrix_all_with_video['c'].sum()

39842

In [369]:
matrix_all_with_video=matrix_all_with_video[matrix_all_with_video['c']>0]

In [371]:
matrix_all_with_video.to_csv('./res/matrix_with_video.csv')

### Блок проверок и визуализации

In [372]:
site=pd.read_csv('sites_coordinate.csv',sep=';')
matrix_all_s=matrix_all_with_video.merge(site,on='site_id',how='left')
matrix_all_s=matrix_all_s.merge(site,left_on='site_id_2',right_on='site_id',how='left')
matrix_all_s=matrix_all_s[['site_id_x','site_id_2','c','ss_lat_x','ss_long_x','ss_lat_y','ss_long_y']]
matrix_all_s.groupby(['site_id_x']).agg({'c':np.sum}).sort_values(by='c', ascending=False)[0:6]

,c
site_id_x,
53,3124
49,2142
14,1427
60,1318
28,1208
126,1097


In [374]:
matrix_all_s= gpd.GeoDataFrame(matrix_all_s,  geometry = [Point(xy) for xy in zip(matrix_all_s['ss_long_y'], matrix_all_s['ss_lat_y'])])
matrix_all_s.crs='epsg:4326'
matrix_all_s=matrix_all_s[matrix_all_s['site_id_x'].isin(['53','49','60','14','28','126'])]
matrix_all_s.groupby(['site_id_x']).apply(lambda x: x.to_file(fr"./res/{x.name}_video.json", index=False, driver="GeoJSON", encoding='utf-8'))
matrix_all2=matrix_all_s[matrix_all_s['site_id_2'].isin(['53','49','60','14','28','126'])]
matrix_all2=matrix_all2[['site_id_2','geometry']].drop_duplicates()
#matrix_all2.to_file(fr"./res/sites.json", index=False, driver="GeoJSON", encoding='utf-8')

## Объединяем с ТАЗами

In [3]:
sites_unic=pd.read_csv('sites_coordinate.csv', sep = ';', encoding='utf-8')

In [8]:
sites_unic[0:1]

,site_id,ss_lat,ss_long
0,0,53.473269,49.47808


In [9]:
sites_gpd = gpd.GeoDataFrame(sites_unic, \
                              geometry = [Point(xy) for xy in zip(sites_unic['ss_long'], sites_unic['ss_lat'])])
sites_gpd.crs='epsg:4326'

In [11]:
transport_zone=gpd.read_file(r'C:\Users\user\Documents\GIT\city\user_scripts\masha_k\Transnavigation\algoritm\557_zone.shp', encoding='utf-8')

In [13]:
site_taz=gpd.sjoin(sites_gpd, transport_zone, how='left', op='within', lsuffix='left', rsuffix='right')

In [14]:
site_taz=site_taz.rename(columns={'NO':'Транспортные районы'})

In [15]:
matrix=pd.read_csv('./res/matrix_Tolyatti.csv')

In [16]:
matrix_taz=matrix.merge(site_taz,on ='site_id', how='left')
matrix_taz=matrix_taz.merge(site_taz,left_on ='site_id_2',right_on ='site_id',how='left')

In [20]:
matrix_taz2=pd.DataFrame(matrix_taz.groupby(['Транспортные районы_x','Транспортные районы_y']).agg({'c':sum}).reset_index())

In [23]:
matrix_taz2.to_csv('matrix_taz_Tolyat.csv',index=False)

In [25]:
matrix_taz2['Транспортные районы_y'].unique()

array([17, 25, 31, 41, 43, 44, 14, 11, 24, 32, 33, 39,  8, 16, 21, 22, 26,
       30, 34, 38, 42, 71, 23, 27, 36, 40, 45,  9], dtype=int64)